# Overview

This notebook uses the raw annotations available in database to prepare a refined dataset and export to csv, which then can be used for model training.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

### Load Data

Next few cells loads the raw data from database, puts them to a dataframe for further analysis and cleaning.

In [3]:
import os
import mysql.connector

conn = mysql.connector.connect(host=os.getenv("MYSQL_HOST") , user=os.getenv("MYSQL_USER"), password=os.getenv("MYSQL_PASSWORD"), database=os.getenv("MYSQL_DB"), auth_plugin='mysql_native_password')

cursor = conn.cursor()

In [4]:
import pandas as pd

def get_all_annotations():
	"""Fetch all annotations from the database."""
	# Define the SQL query to fetch all annotations
	query = """
		SELECT product_id, annotated_image_name, fabric_types, wear_types, colors, textures, value_additions
		FROM aarong.annotations 
		WHERE is_manual_reviewed=1;
	"""

	cursor.execute(query)
	
	rows = cursor.fetchall()
	columns = [i[0] for i in cursor.description]

	df = pd.DataFrame(rows, columns=columns)
	return df

df = get_all_annotations()

In [5]:
df

,product_id,annotated_image_name,fabric_types,wear_types,colors,textures,value_additions
0,1,0560000072957.jpg,Cotton,Saree,"White,Orange",Hand Embroidery,Hand Embroidery
1,1,0560000072957.jpg___dress.png,Cotton,Saree,"White,Orange",Hand Embroidery,Hand Embroidery
2,1,0560000072957_2.jpg,Cotton,Saree,Strawberry Red,Hand Embroidery,Hand Embroidery
3,1,0560000072957_2.jpg___dress.png,Cotton,Saree,Strawberry Red,Hand Embroidery,Hand Embroidery
4,1,0560000072957_1.jpg___dress.png,Cotton,Saree,"White,Orange",Hand Embroidery,Hand Embroidery
...,...,...,...,...,...,...,...
3296,7657,0470000103620.jpg,Silk,Scarves,"blue,orange,white",hand paint,
3297,1277,0550000137483.jpg,Muslin,Saree,"green,yellow","dye,embroidery,printed",embroidery
3298,1387,0550000148658.jpg,Muslin,Saree,"red,white","embroidery,printed",embroidery
3299,7741,0470000103105.jpg,Silk,Scarves,"orange,pink,white",hand paint,


Next I'll implement a few utility functions that might come handy for repeated analysis of columns, such as- getting the label distribution, number of samples with multiple labels, etc.

Another function is used to remove certain labels from a column, which will be useful when there are rare classes that need to be removed upfront.

_NOTE: I have moved the implementations to `my_utils.dataset_utils.py` to be able to reuse it in other notebooks._

In [6]:
from my_utils.dataset_utils import get_multilabel_counts, get_label_distribution, get_rare_labels, remove_labels

## Fabric Type Labels

In [7]:
from IPython.display import display

fabric_types = get_label_distribution(df['fabric_types'])
fabric_num_labels = get_multilabel_counts(df['fabric_types'])

print("Total Unique Fabric Types: ", len(fabric_types))

print("\nFabric Types: ")
display(fabric_types)

print(f"\nMulti-label Items: {fabric_num_labels}")

Total Unique Fabric Types:  37

Fabric Types: 


,count
Cotton,1072
Viscose,604
Viscott (Viscose & Cotton),598
Silk,461
Voile,233
Muslin,137
Handloom Viscose,103
Endi Silk,71
Linen,59
Nakshi Kantha,44



Multi-label Items: {1: 2944, 2: 321, 3: 36}


As there are some fabric types with very few samples, and most of them being a specific category of fabric that can be put into a more general category, I will prepare a mapping from rare fabric types to an appropriate general fabric category.

In [14]:
fabric_mapping = {
		"Cotton": "Cotton",
		"Handloom Cotton": "Cotton",
		"Vortex-Cotton": "Cotton",
		"Vortex Cotton": "Cotton",
		"Ramie Cotton": "Cotton",
		"Mixed Cotton": "Cotton",
		"Cottray (Cotton & Rayon)": "Cotton",
		"Single Jersey Knit Cotton": "Cotton",
		"Muslin": "Muslin",
		"Endi Muslin": "Muslin",
		"Mixed Muslin (Muslin & Endi)": "Muslin",
		"Blended Muslin (Muslin & Silk)": "Muslin",
		"Mixed Muslin": "Muslin",
		"Silk": "Silk",
		"Soft Silk": "Silk",
		"Dupion Silk": "Silk",
		"Joysree Silk": "Silk",
		"Katan": "Silk",
		"Endi Silk": "Silk",
		"Endi Silk-Muslin": "Silk",
		"Half Silk (Silk & Cotton)": "Silk",
		"Mixed Silk (Silk & Cotton)": "Silk",
		"Jamdani Half Silk (Silk & Cotton)": "Silk",
		"SICO (Silk & Cotton)": "Silk Blend",
		"Mixed Silk": "Silk",
		"Viscose": "Viscose",
		"Jacquard Viscose": "Viscose",
		"Handloom Viscose": "Viscose",
		"Mixed Viscose": "Viscose Blend",
		"Viscott (Viscose & Cotton)": "Viscose Blend",
		"Linen": "Linen",
		"Satin": "Satin",
		"Satin Cotton": "Satin Blend",
		"Georgette": "Muslin",
		"Poplin": "Poplin",
		"Denim": "Denim",
		"Mixed Vortex": "Blended Fabric",
		"Addi Cotton": "Blended Fabric",
		"Cashmilon": "Synthetic",
		"Voile": "Voile",
		"Wool": "Wool",
	}

def remap_single_label(labels, mapping):
	"""Remap a single label based on the mapping dictionary."""
	labels = labels.split(",")

	for label in labels:
		if label in mapping:
			return mapping[label]
		
	print(f"Label '{label}' not found in mapping.")
	#return labels[0], labels
	return labels[0]

In [15]:
df['fabric_types_clean'] = df['fabric_types'].apply(lambda x: remap_single_label(x, fabric_mapping))

print(f"Total Fabric Types: {len(df['fabric_types_clean'].unique())}")

df['fabric_types_clean'].value_counts()

Total Fabric Types: 10


fabric_types_clean
Cotton            1069
Viscose            627
Viscose Blend      597
Silk               553
Voile              233
Muslin             150
Linen               59
Synthetic            6
Blended Fabric       4
Poplin               3
Name: count, dtype: int64

So the **fabric_types_clean** is the feature column for fabric type of samples, containing **single label** for each sample. There are some classes that have very few samples, they will be cleaned up later based on a threshold, after working through the rest of the feature inspection.

## Wear Type Labels

In [16]:
print(f"Total Wear Types: {len(df['wear_types'].unique())}")

df['wear_types'].value_counts()

Total Wear Types: 12


wear_types
Shalwar Kameez     856
Saree              795
Panjabi            437
Scarves            269
Nightwear          266
Kurta              224
Maternity          172
Shawls             121
Tops                70
Skirts              43
Coats & Jackets     41
Fabric (Metres)      7
Name: count, dtype: int64

There is no sample with multiple wear type labels, so this can be considered as a **single label** classification problem. Let's put the wear type labels in a separate column **wear_types_clean** for consistency.

In [17]:
df['wear_types_clean'] = df['wear_types'].copy()

## Value Addition Labels


In [18]:
df['value_additions'].value_counts()

value_additions
                                 1284
embroidery                       1120
embroidery,tassels                129
embroidery,applique                92
embroidery,sequins                 78
lace                               76
handloom,fringe                    73
embroidery,handloom                55
handloom                           50
embroidery,nakshi                  44
baluchari,tangail                  37
tangail                            34
fringe                             33
embroidery,lace                    27
Machine Embroidery                 24
nakshi                             22
Screen Print                       18
embroidery,mirror work             18
Block Print                        13
tassels                            11
nakshi,tangail                      8
embroidery,fringe                   6
Hand Embroidery                     5
embroidery,handloom,fringe          5
applique                            4
embroidery,sequins,applique       

This feature possibly need to be multi-label as a number of samples seem to have multiple labels.

Also the "embroidery" value addition seem to exist in almost all samples, so this is a less useful feature and can be dropped from the feature set.

In [19]:
df['value_additions_clean'] = df['value_additions'].apply(lambda x: remove_labels(x, ["embroidery", "Machine Embroidery"]))

df['value_additions_clean'].value_counts()

value_additions_clean
                             2428
tassels                       140
handloom                      105
lace                          103
applique                       96
sequins                        81
handloom,fringe                78
nakshi                         66
fringe                         39
baluchari,tangail              37
tangail                        34
mirror work                    20
Screen Print                   18
Block Print                    13
nakshi,tangail                  8
stone work                      6
Hand Embroidery                 5
sequins,applique                4
handloom,sequins                4
tassels,applique                4
baluchari,tassels,tangail       3
lace,mirror work                2
lace,stone work                 2
handloom,applique               2
mirror work,fringe              2
Printed                         1
Name: count, dtype: int64

So **value_addition_clean** is the feature column for value addition of samples, a **multi-label** feature column.

Now lets see how many unique value additions are there across samples.

In [20]:
value_additions = get_label_distribution(df['value_additions_clean'])

display(value_additions)

print(f"Total Unique Value Additions: {len(value_additions)}")

print(f"multi-label items: {get_multilabel_counts(df['value_additions_clean'])}")

,count
,2428
handloom,189
tassels,147
fringe,119
lace,107
applique,106
sequins,89
tangail,82
nakshi,74
baluchari,40


Total Unique Value Additions: 16
multi-label items: {1: 3155, 2: 143, 3: 3}


So these can be considered potential classes for the value addition feature. As discussed before, the classes with low number of samples will be cleaned up in a step further down below. 

## Texture labels

In [21]:
textures = get_label_distribution(df['textures'])

print(f"Total Unique Textures: {len(textures)}")
print(f"multi-label items: {get_multilabel_counts(df['textures'])}")

textures

Total Unique Textures: 29
multi-label items: {1: 1494, 2: 1182, 3: 548, 4: 59, 5: 18}


,count
embroidery,1596
printed,1443
block print,501
textured,309
screen print,282
tie dye,252
,239
hand paint,226
dobby,108
applique,106


A good number of samples have multiple texture labels, so this is a **multi-label** classification problem. Let's put the texture labels in a separate column **texture_clean** for consistency.

In [22]:
df['textures_clean'] = df['textures'].copy()

## Color Labels

In [23]:
colors = get_label_distribution(df['colors'])
print(f"Total Unique Colors: {len(colors)}")
print(f"multi-label items: {get_multilabel_counts(df['colors'])}")

colors

Total Unique Colors: 35
multi-label items: {1: 327, 2: 1088, 3: 1233, 4: 496, 5: 126, 6: 22, 7: 9}


,count
white,1090
green,999
blue,940
brown,770
pink,746
black,734
red,733
yellow,532
golden,506
orange,495


Some colors seem to be duplicate due to case sensitivity, let's convert all to lower case.

In [24]:
# Convert to lowercase, and remove if there is duplicate due to case sensitivity

def clean_colors(colors):
	"""Convert colors to lowercase and remove duplicates."""
	colors = colors.split(",")
	colors = [color.strip().lower() for color in colors]
	return ",".join(set(colors))


df['colors_updated'] = df['colors'].apply(lambda x: clean_colors(x))


clean_colors = get_label_distribution(df['colors_updated'])
print(f"Total Unique Colors after Cleaning: {len(clean_colors)}")
print(f"multi-label items: {get_multilabel_counts(df['colors_updated'])}")

clean_colors

Total Unique Colors after Cleaning: 24
multi-label items: {1: 327, 2: 1088, 3: 1233, 4: 496, 5: 126, 6: 22, 7: 9}


,count
white,1093
green,999
blue,944
brown,770
pink,759
red,746
black,734
yellow,539
golden,524
orange,520


There are still too many colors, and some of them seem to be rare i.e. in very few samples. Let's map rare colors to a visually similar but more common color.

In [25]:
color_mapping = {
	'magenta': 'red',
	'maroon': 'red',
	'beige': 'white',
	'strawberry red': 'red',
	'silver': 'grey',
	'coffee': 'brown',
	'peach': 'orange',
	'olive': 'green',
	'teal': 'green'
}

def remap_colors(colors, mapping, invalid_colors=['wax dyed', 'multicolour']):
	"""Remap colors based on the mapping dictionary, as well as remove dups if any"""
	colors = colors.split(",")
	colors = [color.strip().lower() for color in colors]

	for i, color in enumerate(colors):
		if color in mapping:
			colors[i] = mapping[color]
	
	# Remove invalid colors
	for invalid_color in invalid_colors:
		if invalid_color in colors:
			colors.remove(invalid_color)

	return ",".join(set(colors))

df['colors_clean'] = df['colors_updated'].apply(lambda x: remap_colors(x, color_mapping))

In [26]:
clean_colors = get_label_distribution(df['colors_clean'])
print(f"Total Unique Colors after Cleaning: {len(clean_colors)}")
print(f"multi-label items: {get_multilabel_counts(df['colors_clean'])}")

clean_colors

Total Unique Colors after Cleaning: 13
multi-label items: {1: 345, 2: 1110, 3: 1231, 4: 465, 5: 124, 6: 19, 7: 7}


,count
white,1093
green,1007
blue,944
red,841
brown,772
pink,759
black,734
yellow,539
golden,524
orange,520


Now this is more addressable size of classes for the scope of this project.

So the **color_clean** is the feature column for color of samples, as a **multi-label** feature.

Let's export these pre-processed features into a csv file so that it can be used as a dataset for model training.

In [29]:
df_dataset = df[['product_id', 'annotated_image_name', 'fabric_types_clean', 'wear_types_clean', 'colors_clean', 'textures_clean', 'value_additions_clean']].copy()

df_dataset.sample(10)

,product_id,annotated_image_name,fabric_types_clean,wear_types_clean,colors_clean,textures_clean,value_additions_clean
2862,7993,0460000029087.jpg___full.png,Voile,Nightwear,"red,brown,white","floral,screen print",lace
3160,7625,0470000103702.jpg,Silk,Scarves,"blue,white",hand paint,
2386,6437,1190000010348.jpg___full.png,Viscose Blend,Kurta,"orange,pink","embroidery,printed",sequins
2920,2425,1420000166671.jpg,Voile,Shalwar Kameez,"brown,black,white","embroidery,printed",
1017,181,0560000070406.jpg,Cotton,Saree,"blue,green","applique,embroidery,floral",applique
23,7,0560000069137.jpg,Cotton,Saree,red,"Machine Embroidery,Floral",Printed
515,286,0560000070516_2.jpg,Cotton,Saree,"orange,red,brown,yellow","embroidery,printed",
3059,7703,0470000103534.jpg,Silk,Scarves,"white,purple,cyan,green",hand paint,
761,259,0560000073403_1.jpg,Cotton,Saree,"red,yellow,green",,tangail
1620,923,0560000071705_2.jpg,Cotton,Saree,"red,pink,green","nakshi,striped",nakshi


In [30]:
# Save to file

df_dataset.to_csv("../../data/aarong_dataset_clean_final.csv", index=False)